<a href="https://colab.research.google.com/github/jdavidleon/number-recognition-network/blob/main/network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this case we will create a neural network to understand how it works. with simple concepts and understanding what is going on.